In [1]:
#from module_1.homework import response
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-04 22:31:10--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-04 22:31:10 (24.0 MB/s) - ‘minsearch.py.4’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [2]:
documents

NameError: name 'documents' is not defined

In [27]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
#SELECT * WHERE course = 'data-engineering-zoomcamp';

In [9]:
index.fit(documents)

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()


In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict= {'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results


In [13]:
def build_prompt (query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION. If you cannot find a direct answer then try to formulate a response based on what is relevant in the CONTEXT.

If the CONTEXT completely doesnt contain the answer or anything similar, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [14]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [16]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    #print('_________Prompt_________')
    #print(prompt)
    #print('_________answer_________')
    #print(answer)
    #print('_________search_results_________')
    #print(search_results)

    return answer


"Yes, you can still enroll after the course has started. You are eligible to submit the homework even if you haven't registered. However, be mindful of the deadlines for turning in the final projects, so don't leave everything for the last minute."

In [17]:
rag('The course has already started, can i still enroll?')


'To run the Kafka Python producer, follow these steps:\n\n1. **Create a Virtual Environment**: This should be done preferably in the directory where your `producer.py` file is located.\n\n   ```\n   python -m venv env\n   ```\n\n2. **Activate the Virtual Environment**: The command varies slightly depending on your operating system.\n\n   - On MacOS and Linux:\n     ```\n     source env/bin/activate\n     ```\n\n   - On Windows:\n     ```\n     env\\Scripts\\activate\n     ```\n\n3. **Install Dependencies**: Make sure to have a `requirements.txt` file that includes all necessary dependencies, then use the following command to install them.\n\n   ```\n   pip install -r ../requirements.txt\n   ```\n\n4. **Run Docker Services**: Ensure that all necessary Docker images and services are up and running before executing your Python file.\n\n   ```\n   docker-compose up\n   ```\n\nFollowing these steps will prepare your environment to run the Kafka Python producer. If you encounter a `KafkaTime

In [19]:
rag('How do I run the Kafka Python producer?')


{'name': '2053dda8ab2a', 'cluster_name': 'docker-cluster', 'cluster_uuid': '31iMmRKuSqSoH8B8JVF2ow', 'version': {'number': '7.17.13', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b211dbb8bfdecaf7f5b44d356bdfe54b1050c13', 'build_date': '2023-08-31T17:33:19.958690787Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [20]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    "http://localhost:9200"
)

print(es_client.info())


/var/folders/_r/_94p_t6s1933ynymfxl25_7h0000gn/T/ipykernel_17962/805618838.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name  = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)


/opt/homebrew/Caskroom/miniforge/base/envs/llm_zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from tqdm.auto import tqdm

100%|██████████| 948/948 [00:03<00:00, 269.10it/s]


In [41]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [42]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

/var/folders/_r/_94p_t6s1933ynymfxl25_7h0000gn/T/ipykernel_17962/3686858327.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [ ]:
query = 'I just discovered the course. Can i still join this cohort?'
elastic_search(query)